In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

In [7]:
train_data = pd.read_csv('data/task42.training.csv')
test_data = pd.read_csv('data/task42.test.WITHOUT.answers.csv')

In [8]:
train_data.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f16,f17,f18,f19,f20,f21,f22,f23,f24,class
0,27.247,5.235,21.148,1.819,18.794,2.827,4.132,10.042,14.238,15.316,...,0.571,-10.479,7.346,-16.383,-16.993,0.095,8.777,9.392,J,I
1,17.007,8.053,19.253,17.910,1.208,2.209,5.740,13.116,14.077,12.902,...,-10.122,-25.352,11.887,-7.805,-22.207,10.932,1.043,2.641,D,F
2,11.627,15.101,15.085,6.266,14.597,2.590,4.340,6.149,12.116,6.769,...,-9.695,-12.643,1.012,2.194,-10.821,9.390,0.328,8.167,D,B
3,7.848,12.820,6.811,2.360,16.185,3.721,1.859,30.002,7.198,19.888,...,-12.928,-18.363,2.534,-7.653,-10.458,0.939,1.643,9.689,I,G
4,6.060,11.139,8.253,6.030,15.321,3.572,4.840,29.867,8.839,19.805,...,6.736,3.794,1.009,6.736,-16.833,22.885,0.210,8.688,C,J


In [9]:
test_data.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24
0,14.661,10.906,13.997,18.789,3.508,3.058,4.422,14.171,8.979,17.216,...,1.869,0.536,-14.941,12.178,-5.446,-2.566,0.977,3.530,6.853,J
1,14.578,7.126,21.684,12.571,13.452,2.394,3.902,9.056,13.781,13.020,...,1.769,-11.948,-21.154,5.587,-8.858,-18.778,6.249,0.946,8.471,G
2,15.877,9.065,12.778,12.995,15.872,1.113,3.961,15.509,10.669,12.952,...,-2.281,-22.332,-15.565,3.520,-9.337,-23.918,11.135,0.562,9.930,F
3,16.208,6.599,20.308,1.905,15.486,1.853,4.391,8.266,13.111,7.480,...,-0.414,0.741,-12.829,9.997,-10.729,-23.497,9.124,1.047,6.316,F
4,12.913,9.889,12.023,13.053,10.181,1.540,4.244,19.819,8.257,21.733,...,2.832,-9.252,-7.171,0.600,-9.252,-4.633,1.985,0.550,9.473,F


In [56]:
my_encoder = LabelEncoder()
# my_class_currency = np.array(my_encoder.fit_transform(train_data['f24'])).reshape(-1,1)
# #Create a "diccionary" to translate the categories into the actual values once you have the output
# my_class_decoder = list(np.unique(train_data['f24']))
process_array = my_encoder.fit_transform(train_data['f24'])
process_array_ans = my_encoder.fit_transform(train_data['class'])

In [60]:
# Split training data for test data
sp_train_data, sp_test_data = train_test_split(train_data, test_size=0.1, random_state=100)

sp_train_data_set = sp_train_data.iloc[:,0:25]

# processing categories data
df1_1 = pd.DataFrame(process_array, columns = ['PROC'])
sp_train_data_set['f24'] = df1_1['PROC']

df1_1ans = pd.DataFrame(process_array, columns = ['PROC'])
sp_train_data['class'] = df1_1ans['PROC']
sp_train_data_ans = sp_train_data['class']

sp_test_data_ans = sp_test_data['class']
sp_test_data_set = sp_test_data.iloc[:,0:25]

df1_2 = pd.DataFrame(process_array, columns = ['PROC'])
sp_test_data_set['f24'] = df1_2['PROC']

# process_array2 = my_encoder.fit_transform(sp_test_data_set['class'])
# df1_3 = pd.DataFrame(process_array2, columns = ['PROC'])
# test_data['class'] = df1_3['PROC']
# test_data.head()
sp_train_data_ans

<ipython-input-60-e03983516af4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp_train_data['class'] = df1_1ans['PROC']


19249    0
6912     4
11263    6
26710    8
5710     7
        ..
12119    5
24423    8
14147    5
23320    6
5640     0
Name: class, Length: 27000, dtype: int32

In [61]:
print(len(sp_train_data))
print(len(sp_test_data))

27000
3000


In [94]:
def model_1():
    # create model
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(10, input_dim=2, activation='relu'))
    model.add(tf.keras.layers.Dense(5, activation='relu'))
    model.add(tf.keras.layers.Dense(5, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [95]:
model1_1 = model_1()

In [96]:
model1_1.fit(sp_train_data_set, sp_train_data_ans, epochs=500, batch_size=200, verbose=1, shuffle=True)

Epoch 1/500


ValueError: in user code:

    C:\Users\FrankLIN\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\FrankLIN\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\FrankLIN\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\FrankLIN\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\FrankLIN\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
   

In [136]:
# MLP Training
# hidden_layer_sizes=(23,) = 0.998
# hidden_layer_sizes=(46,) = 0.9992
# hidden_layer_sizes=(100) = 1.0
#1e-05
# mlp = MLPClassifier(alpha=1e-05, hidden_layer_sizes=(25), random_state=1, solver='sgd', max_iter=900000000, verbose=1)
mlp = MLPRegressor(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)
mlp.fit(train_data_set, train_data_ans)
# mlp.score(sp_test_data_set, sp_test_data_ans)

ValueError: could not convert string to float: 'I'

In [95]:
mlp.score(train_data_set, train_data_ans)

0.9324

In [96]:
mlp.loss_

0.17251093184756477

In [118]:
test_x = test_data[['f0','f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18','f19','f20','f21','f22','f23','f24']]

df2 = pd.DataFrame(process_array, columns = ['PROC'])
test_x['f24'] = df2['PROC']

In [119]:
y_pred = output_clf.predict(test_x)

In [120]:
number = np.arange(1,len(y_pred)+1)

In [121]:
sample = pd.read_csv('data/task42.SAMPLE.submission.csv')
sample.head()

,id,Predicted
0,1,A
1,2,B
2,3,C
3,4,D
4,5,E


In [122]:
submission = {'Id':number, 'Predicted':y_pred}
df = pd.DataFrame(submission,  columns= ['Id', 'Predicted'])

In [123]:
df.to_csv("submission.csv", index = False, header=True)